## Multiple Imputation by Chained Equation(MICE)

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('Retail_data.csv')

In [3]:
pd.set_option("display.max_columns",500)

In [4]:
df.sample(3)

,hour,order_id,customer_id,product_id,product_price,variant_id,orders,gross_sales,discounts,returns,net_sales,shipping,taxes,total_sales,ordered_item_quantity,net_quantity,returned_item_quantity,order_name,product_type,product_title,product_vendor,variant_title,api_client_title
22016,22-08-2021 15:00,4.024380e+12,5.510250e+12,6.751540e+12,2.99,4.005880e+13,1,5.48,0.0,0.0,5.48,0.0,0.50,5.98,1,1,0,#5191,Flour & Atta,Daily Delight - Periyar- Roasted Rava,Periyar,1 kg,Point of Sale
26161,10-09-2021 23:00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
35547,15-10-2021 18:00,4.208920e+12,5.269060e+12,6.763900e+12,3.89,4.010340e+13,1,7.14,0.0,0.0,7.14,0.0,0.64,7.78,1,1,0,#7952,Sweets & Snacks,Mithai Sweets - Ladoo,Mithai,450 g,Point of Sale


In [5]:
df=df.drop(['hour'], axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101929 entries, 0 to 101928
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                101929 non-null  float64
 1   customer_id             101929 non-null  float64
 2   product_id              101929 non-null  float64
 3   product_price           101929 non-null  float64
 4   variant_id              101929 non-null  float64
 5   orders                  101929 non-null  int64  
 6   gross_sales             101929 non-null  float64
 7   discounts               101929 non-null  float64
 8   returns                 101929 non-null  float64
 9   net_sales               101929 non-null  float64
 10  shipping                101929 non-null  float64
 11  taxes                   101929 non-null  float64
 12  total_sales             101929 non-null  float64
 13  ordered_item_quantity   101929 non-null  int64  
 14  net_quantity        

The number of categories is quite large as one-hot encoding can lead to high memory consumption. When the order does not matter in categorical feature.

In [7]:
#drop the rows that has NaN values in following columns
df=df.dropna(subset=['order_name','product_type','product_title','product_vendor','variant_title','api_client_title'], how='all')

In [8]:
df.isnull().sum()

order_id                      0
customer_id                   0
product_id                    0
product_price                 0
variant_id                    0
orders                        0
gross_sales                   0
discounts                     0
returns                       0
net_sales                     0
shipping                      0
taxes                         0
total_sales                   0
ordered_item_quantity         0
net_quantity                  0
returned_item_quantity        0
order_name                    0
product_type              16900
product_title               522
product_vendor            16405
variant_title             25355
api_client_title              0
dtype: int64

In [9]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

#list of categorical variables
var= ['order_name','product_type','product_title','product_vendor','variant_title','api_client_title']

#This function will encode non-null data and replace it in the original data
def ordinalencode(x):
    nonulls = np.array(x.dropna())                                 #retains only non-null values
    impute_reshape = nonulls.reshape(-1,1)                         #reshapes the data for encoding
    impute_ordinal = encoder.fit_transform(impute_reshape)         #encode data
    x.loc[x.notnull()] = np.squeeze(impute_ordinal)                #Assign back encoded values to non-null values
    return x

#encoding all the categorical data in the data set through for loop

for i in var:
    ordinalencode(df[i])

In [10]:
df.head(3)

,order_id,customer_id,product_id,product_price,variant_id,orders,gross_sales,discounts,returns,net_sales,shipping,taxes,total_sales,ordered_item_quantity,net_quantity,returned_item_quantity,order_name,product_type,product_title,product_vendor,variant_title,api_client_title
11,3.824510e+12,5.268530e+12,5.931930e+12,0.99,3.705210e+13,1,1.82,-0.18,0.0,1.64,0.0,0.14,1.78,1,1,0,782.0,5.0,2198.0,52.0,19.0,2.0
12,3.824510e+12,5.268530e+12,5.931930e+12,1.39,3.705210e+13,1,2.54,-0.24,0.0,2.30,0.0,0.20,2.50,1,1,0,782.0,9.0,7665.0,155.0,8.0,2.0
13,3.824510e+12,5.268530e+12,6.723260e+12,0.79,3.994350e+13,1,1.30,-0.12,0.0,1.18,0.0,0.24,1.42,1,1,0,782.0,2.0,4568.0,173.0,NaN,2.0


The MICE algorithm can impute mixes of continuous, binary, unordered categorical and ordered categorical data

In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [12]:
lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman',random_state=0)
X=imp.fit_transform(df)

[IterativeImputer] Completing matrix with shape (95968, 22)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.89
[IterativeImputer] Change: 1761.900832390595, scaled tolerance: 43049600000.0 
[IterativeImputer] Early stopping criterion reached.


In [13]:
df = pd.DataFrame(X,columns=df.columns)

In [14]:
df.shape

(95968, 22)